In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf

# from google.colab import drive

# Splitting for training model

In [ ]:
def preprocess_image(img_path, target_size=(224, 224)):

    img = cv2.imread(img_path)
    img = cv2.resize(img, target_size)
    img = img.astype('float32') / 255.0
    return img

# Google drive (if colab)
# base_folder = '/content/drive/My Drive/ConsumerCompass/data'

base_folder = 'data 2' # insert your path

folders = [os.path.join(base_folder, folder_name) for folder_name in ['healthy', 'unhealthy', 'glamorous', 'drab', 'rugged', 'gentle', 'fun', 'dull',
                                                                      'innovation', 'conservatism', 'premuim', 'accessibility', 'minimalism', 'detail', 'safety', 'risk']]

#class lables 
label_map = {'healthy': 0, 'unhealthy': 1, 'glamorous': 2, 'drab': 3, 'rugged': 4, 'gentle': 5, 'fun': 6, 'dull': 7,
             'innovation': 8, 'conservatism': 9, 'premuim': 10, 'accessibility': 11, 'minimalism': 12, 'detail': 13, 'safety': 14, 'risk': 15}
all_images = []
all_labels = []

# Dataframe creation 
for folder in folders:
    label_index = label_map[os.path.basename(folder)]
    print(folder)
    for img_file in os.listdir(folder):
        img_path = os.path.join(folder, img_file)
        if os.path.isfile(img_path):
            img = preprocess_image(img_path)
            label_vector = [0] * 16 # number of lables
            label_vector[label_index] = 1
            all_images.append(img)
            all_labels.append(label_vector)

all_images = np.array(all_images)
all_labels = np.array(all_labels)

train_images, test_images, train_labels, test_labels = train_test_split(all_images, all_labels, test_size=0.3, random_state=42)
val_images, test_images, val_labels, test_labels = train_test_split(test_images, test_labels, test_size=0.5, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    # Conv1
    Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)),
    # MaxPool1
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    # Norm1
    BatchNormalization(),

    # Conv2
    Conv2D(filters=256, kernel_size=(5, 5), padding='same', activation='relu'),
    # MaxPool2
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    # Norm2
    BatchNormalization(),

    # Conv3
    Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu'),

    # Conv4
    Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu'),

    # Conv5
    Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'),
    # MaxPool5
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),

    Flatten(),

    # FC6
    Dense(4096, activation='relu'),
    # Dropout FC6
    Dropout(0.5),

    # FC7
    Dense(4096, activation='relu'),
    # Dropout FC7
    Dropout(0.5),

    # FC8
    Dense(16, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


#fit the model
history = model.fit(
    train_images,
    train_labels,
    validation_data=(val_images, val_labels),
    epochs=10
)

test_loss, test_accuracy = model.evaluate(test_images, test_labels)

model.save('/content/model_multilabel_brandimagenet')

In [ ]:
from tensorflow.keras.metrics import BinaryAccuracy

predictions = model.predict(test_images)

accuracy_metrics = []
for i in range(0, predictions.shape[1], 2):
    metric = BinaryAccuracy()
    metric.update_state(test_labels[:, i:i+2], predictions[:, i:i+2])
    accuracy_metrics.append(metric.result().numpy())

class_pairs = ['healthy-unhealthy', 'glamorous-drab', 'rugged-gentle', 'fun-dull',
               'innovation-conservatism', 'premuim-accessibility', 'minimalism-detail', 'safety-risk']

#print accuracy for every single pair
for i, acc in enumerate(accuracy_metrics):
    print(f"Accuracy for {class_pairs[i]}: {acc}")

#print avg accuracy 
average_accuracy = np.mean(accuracy_metrics)
print(f"Average Accuracy across all class pairs: {average_accuracy}")

In [ ]:
def calculate_tp_fp_tn_fn_per_pair(model, test_images, test_labels):
    predictions = model.predict(test_images)
    predictions_binary = (predictions > 0.5).astype(int)
    
    results = []

    for i in range(0, test_labels.shape[1], 2):
        # filtering images for every pair
        relevant_indices = (test_labels[:, i] + test_labels[:, i + 1] > 0)
        relevant_labels = test_labels[relevant_indices, i:i+2]
        relevant_predictions = predictions_binary[relevant_indices, i:i+2]

        tp_metric = tf.keras.metrics.TruePositives()
        fp_metric = tf.keras.metrics.FalsePositives()
        tn_metric = tf.keras.metrics.TrueNegatives()
        fn_metric = tf.keras.metrics.FalseNegatives()
        
        #calculate tp fp tn fn for inside pair
        tp_metric.update_state(relevant_labels, relevant_predictions)
        fp_metric.update_state(relevant_labels, relevant_predictions)
        tn_metric.update_state(relevant_labels, relevant_predictions)
        fn_metric.update_state(relevant_labels, relevant_predictions)
        
        results.append({
            'TP': tp_metric.result().numpy(),
            'FP': fp_metric.result().numpy(),
            'TN': tn_metric.result().numpy(),
            'FN': fn_metric.result().numpy()
        })
        
    return results

results = calculate_tp_fp_tn_fn_per_pair(model, test_images, test_labels)

In [ ]:
# function to calculate precision for pair
def calculate_precision(results):
    precisions = []
    for result in results:
        tp = result['TP']
        fp = result['FP']
        if tp + fp == 0:
            precision = 0
        else:
            precision = tp / (tp + fp)
        precisions.append(precision)
    return precisions

#calculate precision for every pair
precisions = calculate_precision(results)

#calculate and print avg precision
average_precision = np.mean(precisions)
print(f"Average Precision across all class pairs: {average_precision:.4f}")

In [ ]:
# from tensorflow.keras.models import load_model

# model = load_model('/content/model2')

# image_path = '/content/test/pic1.jpg' # upload to google collab manually

# img = preprocess_image(image_path)

# img_batch = np.expand_dims(img, axis=0)

# predictions = model.predict(img_batch)

# final_classification = []

# for i in range(0, len(predictions[0]), 2):
#     if predictions[0][i] > predictions[0][i+1]:
#         final_classification.extend([1, 0])
#     else:
#         final_classification.extend([0, 1])

# print(final_classification)

In [ ]:
# queries = ["healthy", "unhealthy", "glamorous", "drab",
#            "rugged", "gentle", "fun", "dull",
#            'innovation', 'conservatism', 'premuim', 'accessibility',
#            'minimalism', 'detail', 'safety', 'risk']

# def normalize_and_print_predictions(predictions):
#     normalized_predictions = []

#     for i in range(0, len(predictions[0]), 2):
#         pair_sum = predictions[0][i] + predictions[0][i+1]
#         normalized_pair = [predictions[0][i] / pair_sum, predictions[0][i+1] / pair_sum]
#         normalized_predictions.extend(normalized_pair)

#     # Print the normalized predictions
#     for i in range(0, len(normalized_predictions), 2):
#         print(f"{queries[i]}: [{normalized_predictions[i]:.2f}, {normalized_predictions[i+1]:.2f}]")

# normalize_and_print_predictions(predictions)